# Covid-19 Death Rates
> xyz

- toc: true
- branch: master
- comments: true
- categories: [fastpages, jupyter]
- images: images/test.png

In [3]:
# hide
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import altair as alt

from IPython.display import HTML, display
import pytz

import ipywidgets as widgets

from datetime import datetime, timedelta, timezone

print('Libraries successfully imported.')

Libraries successfully imported.


In [4]:
#hide
url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
url_2 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv'
url_3 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/8a88fe6d-d8fb-41a3-9d04-f0550a44999f/download/daily_change_in_cases_by_phu.csv'

src_conpos = pd.read_csv(url, index_col=0, parse_dates=['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']).reset_index()
src_testing = pd.read_csv(url_2, index_col=0, parse_dates=['Reported Date']).reset_index()
src_daily = pd.read_csv(url_3, index_col=0, parse_dates=['Date']).reset_index()

# duplicate entry for December 5th, 2020 causing error
src_testing.drop_duplicates(inplace=True)

print('Source files successfully loaded.')

Source files successfully loaded.


In [5]:
#hide_input
df_deathts = src_conpos[src_conpos['Outcome1'] == 'Fatal'].groupby(['Case_Reported_Date', 'Age_Group']).count()['Row_ID'] \
.reset_index().rename(columns={"Row_ID": "Cases"})

fig_deaths = px.bar(df_deathts,
                    y='Cases',
                    x='Case_Reported_Date',
                    color='Age_Group',
                    labels={"Case_Reported_Date" : "Reported Date", "Age_Group" : "Age Group", "Cases" : "Deaths"},
                    color_discrete_sequence=px.colors.qualitative.Set2,
                    title = "Ontario Covid-19 Deaths by Age Group"
)

fig_deaths.show()

In [6]:
#hide_input
df_deaths = src_conpos[src_conpos['Outcome1'] == 'Fatal'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Deaths'})

df_res = src_conpos[src_conpos['Outcome1'] == 'Resolved'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Resolved'})

df_nres = src_conpos[src_conpos['Outcome1'] == 'Not Resolved'].groupby(['Age_Group']).count()["Row_ID"] \
.reset_index().iloc[:,[0,1]].rename(columns={'Row_ID': 'Not Resolved'})

# merge dataframes together and clean up

df_age = df_deaths.merge(df_res, on='Age_Group').merge(df_nres, on='Age_Group').replace('<20', '0-19') \
.rename(columns={'Age_Group': 'Age Group'}).sort_values('Age Group')

del [df_deaths, df_res, df_nres]

# include percentages
df_age['Death Rate'] = (df_age['Deaths'] / (df_age['Resolved'] + df_age['Not Resolved']))
df_age['Survival Rate'] = (1 - df_age['Death Rate'])

x = df_age \
.drop(columns=['Resolved', 'Not Resolved']) \
.style.format({"Death Rate": "{:.2%}",
                     "Survival Rate": "{:.2%}"}) \
                     .background_gradient(cmap='Reds', subset=['Deaths', 'Death Rate']) \
                     .background_gradient(cmap='Greens', subset=['Survival Rate']) \
                     .hide_index()

x.set_properties(**{'text-align': 'center'}).hide_index()

Age Group,Deaths,Death Rate,Survival Rate
0-19,1,0.00%,100.00%
20s,6,0.02%,99.98%
30s,11,0.04%,99.96%
40s,33,0.15%,99.85%
50s,134,0.58%,99.42%
60s,362,2.49%,97.51%
70s,745,10.18%,89.82%
80s,1529,26.04%,73.96%
90s,1367,40.48%,59.52%
